In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split

# from kmodes.kprototypes import KPrototypes

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance
from kneed import KneeLocator

from sklearn.decomposition import PCA


from tqdm import tqdm
import sys
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_raw = pd.read_csv('../country_data_clustering/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df = df_raw.copy()

In [ ]:
############## Data exploration & Data Cleaning ###################

In [ ]:
#check for missing value
df.isnull().values.any()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#look at the distribution
for x in df.columns:
    fig = px.histogram(df[x],title=x)
    # fig.update_title(x)
    fig.show()

In [ ]:
#looking at the type of the column
df.TotalCharges.dtypes

In [ ]:
#deal with the empty total charges
df = df[df['TotalCharges'] != ' ']

In [ ]:
#separating the prices in bins
df['TotalCharges'] = pd.cut(df.TotalCharges.astype('float64'), 10,labels=[1,2,3,4,5,6,7,8,9,10])

In [ ]:
#second look at the distribution
fig = px.histogram(df['TotalCharges'],title='Total Charges')
fig.show()


In [ ]:
# #deal with the empty total charges
# df = df[df['TotalCharges'] != ' ']

In [ ]:
############## Data preprocessing & feature engineering #####################

In [ ]:
df.info()

In [ ]:
# The k-Means and Mean Shift algorithms can only process numerical data. Therefore, we will transform source data that are not float values to integers to make them mathematically digestible.

#type conversions

df['tenure'] = df['tenure'].astype(np.float64) # -> Columns to which we want to apply aggregation functions such as sum or mean
df['TotalCharges'] = df['TotalCharges'].astype(np.float64)
df['SeniorCitizen'] = df['SeniorCitizen'].astype(object) # ->Columns that will not be subject to computations, apart from counting their elements, in preparation for the upcoming conversion steps.

In [ ]:
df.info()

In [ ]:
# visualize the unique categorical element per column
print(df.select_dtypes('object').nunique())

# stats on the numerical variables
df.select_dtypes(exclude='object').describe()

In [ ]:
df1 = df.drop(['customerID'],axis=1).copy()

In [ ]:
#scale the numerical values -> so the model doesn't get trapped by large differences between the column value
numcols = list(df1.dtypes[df1.dtypes=='float64'].index)
print(numcols)
scaler = StandardScaler()
df1[numcols] = scaler.fit_transform(df1[numcols])
df1.describe()

In [ ]:
# helper function: translate category column to numerical column
def catcode(df, col): 
    df[col + "_n"] = df[col].astype("category").cat.codes
    df = df.drop(col, axis=1, inplace=True)

# create numerical columns from categories
_ = [catcode(df1, col) for col in list(df1.dtypes[df1.dtypes == np.object].index)]
print(df1.select_dtypes(exclude="object").nunique())
df1.describe()

In [ ]:
# principal components
pca = PCA(n_components=12)
res_pca = pca.fit_transform(df1)

# scree plot
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color="blue")
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)

df1_pca = pd.DataFrame(res_pca)

In [ ]:
frTRAIN = 0.8               # % size of training dataset
RNDN = 42                   # random state
nK = 12                     # initial guess: clusters

In [ ]:
# elbow score plot with Yellowbrick
def elbowplot(df, elbowmetric, model):
    print("Elbow Score Plot (" + str(elbowmetric) + " metric):")
    vis = KElbowVisualizer(
        model, 
        k=(2,nK), 
        metric=elbowmetric,
        locate_elbow=True, 
        timings=False)
    vis.fit(df)      
    print("elbow value = optimal k:", f'{vis.elbow_value_:.0f}', \
            " | elbow score:", f'{vis.elbow_score_:,.3f}')
    vis.show()  
    
    
    
# call elbow plot for each of 3 alternative metrics
    # distortion = mean sum of squared distances to center
    # silhouette = mean ratio of intra-cluster and nearest-cluster distance
    # calinski = ratio of within to between cluster dispersion

model = KMeans(random_state=RNDN)
_ = [elbowplot(df1, m, model) for m in tqdm(["distortion", "silhouette", "calinski_harabasz"])]    


In [ ]:
# kmeans: looking for the elbow - compare number of clusters by their inertia scores

# run kMeans for alternative number of clusters k
inertia_scores = [KMeans(
                    n_clusters=k, 
                    init='k-means++', 
                    n_init=10, max_iter=100, random_state=RNDN). \
                    fit(df1).inertia_ \
                    for k in range(2,nK)]


dict_inertia = dict(zip(range(2,nK), inertia_scores))
print("inertia scores (sum of squared errors) by number of clusters:")
_ = [print(k, ":", f'{v:,.0f}') for k,v in dict_inertia.items()]

# scree plot: look for elbow
plt.figure(figsize=[8,5])
plt.plot(range(2,nK), inertia_scores, color="blue")
plt.title("inertia (sum of squared errors) vs. number of clusters")
plt.xticks(np.arange(2,nK,1.0))
plt.xlabel("number of clusters K")
plt.ylabel("inertia");

In [ ]:
# inertia scores: confirm visual clue of elbow plot
# KneeLocator class will detect elbows if curve is convex; if concavem will detect knees
inertia_knee_a3 = KneeLocator(
        range(2,nK), 
        inertia_scores, 
        S=0.1, curve="convex", direction="decreasing")

K_inertia_a3 = inertia_knee_a3.elbow   
print("elbow at k =", f'{K_inertia_a3:.0f} clusters')

In [ ]:
# kMeans: silhouette score
# initial example: silhouette score for 4 clusters
k = 4
model = KMeans(n_clusters=k, random_state=RNDN, verbose=0)
clusters_assigned = model.fit_predict(df1)
K_sil_a3 = silhouette_score(df1, clusters_assigned)
print("silhouette score for", k, "clusters: " f'{K_sil_a3:.3f}')

In [ ]:
# find maximum silhouette score for up to kN clusters
sil_scores = [silhouette_score(
                                df1, 
                                KMeans(n_clusters=k, random_state=RNDN). \
                                fit_predict(df1)) \
                                for k in tqdm(range(2,nK))]

dict_sil = dict(zip(range(2,nK), sil_scores))
print("silhouette scores:")
_ = [print(k, ":", f'{v:,.3f}') for k,v in dict_sil.items()]
K_sil_a3 = max(dict_sil, key=dict_sil.get)            # optimal clusters
sil_opt_a3 = dict_sil[K_sil_a3]                       # optimal silhouette score
print("maximum silhouette score for", f'{K_sil_a3:.0f} clusters: ', f'{sil_opt_a3:.3f}')

plt.figure(figsize=[7,5])
plt.plot(range(2,nK), sil_scores, color="red")
plt.title("silhouette scores vs. number of clusters")
plt.xticks(np.arange(2,nK,1))
plt.xlabel("number of clusters K")
plt.ylabel("silhouette score")
plt.show()

In [ ]:
# silhouette score plots with Yellowbrick
dict_score = dict()
fig, ax = plt.subplots(int(np.ceil(nK/2)-1), 2, figsize=(15,30))

for i in tqdm(range(2,nK)):
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=RNDN)
    
    q, mod = divmod(i, 2)
    vis = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod], is_fitted=False)
    vis.fit(df1)
    vis.finalize()
    dict_score[i] = vis.silhouette_score_


print("silhouette scores for k clusters:")
_ = [print(k,":",f'{v:.3f}') for k,v in dict_score.items()]

K_sil_a3 = max(dict_score, key=dict_score.get)          # optimal clusters
sil_opt_a3 = dict_score[K_sil_a3]                       # optimal (maximal) silhouette score
print("maximum silhouette score for", f'{K_sil_a3:.0f} clusters: ', f'{sil_opt_a3:.3f}')


In [ ]:

# optimal number of clusters: intercluster distances
model = KMeans(
    n_clusters=K_sil_a3, init='k-means++', n_init=10, max_iter=100, random_state=RNDN)
visD = InterclusterDistance(
    model, max_size=20000, legend=False, random_state=RNDN)
visD.fit(df1)
visD.finalize()

In [ ]:
# intercluster distance maps: alternative numbers of clusters
dict_score = dict()
nK = 8
fig, ax = plt.subplots(int(np.ceil(nK/2))-1, 2, figsize=(10,15))

for i in tqdm(range(2,nK)):
    km = KMeans(
                n_clusters=i, 
                init='k-means++', n_init=10, max_iter=100, random_state=RNDN)
    
    q, mod = divmod(i, 2)
    vis = InterclusterDistance(
        km, ax=ax[q-1][mod], max_size=10000, legend=False, random_state=RNDN)
    vis.fit(df1)
    vis.finalize()
    dict_score[i] = vis.scores_

In [ ]:
# %split training vs test dataset
df_train, df_test = train_test_split(df1, train_size=frTRAIN, random_state=RNDN)


# training: generate "Cluster" column based on optimal number of clusters
model = KMeans(n_clusters=5, random_state=RNDN)
res = model.fit_predict(df_train)
df_train.insert(0, "Cluster", res)     # insert cluster labels as new column
df_train.tail()


# training: get silhouette score
sil_train = silhouette_score(df_train, res)
# print("training: silhouette score for", f'{K_sil_a3:.0f} clusters: {sil_train:.3f}')

In [ ]:
df_train

In [ ]:
####################################  Interpretation: Cluster Profiling and Dashboard #############################################

In [ ]:
df1['Clusters'] = model.fit_predict(df1)

In [ ]:
#size of clusters -> to check that there are no obvious outliers or exotic tiny clusters, nor a dominant cluster that dwarfs the others.
df_grp = df1.groupby('Clusters').count()
df_grp

In [ ]:
df['Clusters'] = df1['Clusters']

In [ ]:
pd.crosstab(df['Clusters'],
            df['Churn'],
            values=df['MonthlyCharges'],
            aggfunc='mean',
            normalize=False)

In [ ]:
pd.crosstab(df['Clusters'],
            df['StreamingTV'],
            values=df['StreamingTV'],
            aggfunc='count',
            normalize=False)

In [ ]:
#most frequent column values for each cluster
df.groupby(['Clusters']).agg(lambda x: x.value_counts().index[0])

In [ ]:
# helper function: pie charts for categorical variables
def cluster_pies(df):
    
    # number of categorical variables
    c = len(df.select_dtypes("object").nunique())
    
    # number of clusters
    K = df["Clusters"].nunique()

    for k in tqdm(range(K)):
        dfc = df[df["Clusters"]==k]
        chrg = dfc["MonthlyCharges"].median()
        ten = dfc["tenure"].median()
        cases = dfc.shape[0]

        fig = plt.figure(figsize=(50, 12))
        fig.suptitle("Clusters " + str(k) + ": " + \
            f'{cases:,.0f}' + " cases | " + \
            "median charge " + f'{chrg:.2f}' + \
            " | median tenure " + f'{ten:.0f}')


        ax1 = plt.subplot2grid((2,c),(0,0))
        plt.pie(dfc["Contract"].value_counts(), labels=dfc["Contract"].unique())
        plt.title("Contract");

        ax1 = plt.subplot2grid((2,c),(0,1))
        plt.pie(dfc["gender"].value_counts(), labels=dfc["gender"].unique())
        plt.title("gender");

        ax1 = plt.subplot2grid((2,c),(0,2))
        plt.pie(dfc["SeniorCitizen"].value_counts(), labels=dfc["SeniorCitizen"].unique())
        plt.title("SeniorCitizen");

        ax1 = plt.subplot2grid((2,c),(0,3))
        plt.pie(dfc["Partner"].value_counts(), labels=dfc["Partner"].unique())
        plt.title("Partner");

        ax1 = plt.subplot2grid((2,c),(0,4))
        plt.pie(dfc["PhoneService"].value_counts(), labels=dfc["PhoneService"].unique())
        plt.title("PhoneService");

        ax1 = plt.subplot2grid((2,c),(0,5))
        plt.pie(dfc["InternetService"].value_counts(), labels=dfc["InternetService"].unique())
        plt.title("InternetService");

        ax1 = plt.subplot2grid((2,c),(0,6))
        plt.pie(dfc["StreamingTV"].value_counts(), labels=dfc["StreamingTV"].unique())
        plt.title("StreamingTV");

In [ ]:
def cluster_profile(df):
    dfc = df.groupby("Clusters").agg({ 
        "MonthlyCharges": "median",
        "Contract": lambda x: x.value_counts().index[0],
        "tenure": "median",
        "gender": lambda x: x.value_counts().index[0],
        "SeniorCitizen": lambda x: x.value_counts().index[0],
        "Partner": lambda x: x.value_counts().index[0],
        "Dependents": lambda x: x.value_counts().index[0],
        "PhoneService": lambda x: x.value_counts().index[0],
        "InternetService": lambda x: x.value_counts().index[0],
        "StreamingTV": lambda x: x.value_counts().index[0],
        "PaperlessBilling": lambda x: x.value_counts().index[0],
        "PaymentMethod": lambda x: x.value_counts().index[0],
        "Churn": lambda x: x.value_counts().index[0]
                                    })    #.sort_values(by=["MonthlyCharges"], ascending=False)

    cluster_pies(df)
    return dfc

In [ ]:
cluster_profile(df).T